In [1]:
import numpy as np
import pandas as pd
import xml
import json
import spacy
import pymorphy2
from spacy import displacy
import ru2
import os
import re
import json
import logging
import sys
from collections import defaultdict
import operator


RusLem = pymorphy2.MorphAnalyzer()
nlp = spacy.load('ru2')


def pipe_add(n, names=['sentencizer']):
    pos = 0
    for x in names:
        if not x in n.pipe_names:
            if pos >= len(n.pipe_names):
                n.add_pipe(n.create_pipe(x))
                pos += 1
            else:
                n.add_pipe(n.create_pipe(x), before=n.pipe_names[pos])
                pos += 1
    return n

nlp = pipe_add(spacy.load('./ru2'), ['sentencizer', 'tagger', 'parser'])
print("RU pipeline: {}".format(nlp.pipe_names))

RU pipeline: ['sentencizer', 'tagger', 'parser', 'ner']


In [2]:
replica = 'мама отмыла раму'
doc = nlp(replica)
displacy.render(doc, style='dep', options={'compact':True})

In [3]:
for token in doc:
    print(token, token.pos_, token.lemma_)

мама NOUN мама
отмыла VERB отмыть
раму NOUN рама


In [43]:
RusLem.parse('иметь')[0].tag.POS

'INFN'


## Подготавливаем оценочные фреймы для тестирования

Оставляем только варианты из одного слова и глаголы

In [2]:
evaluation_frames_data = json.load(open("./frames20190730.json", "r", encoding='cp1251'))

In [5]:
evaluation_frames_data_filter = {}
print(f'Всего оценочных фреймов: {len(evaluation_frames_data)}')
all_word_cnt = 0
single_word_cnt = 0
word_set = set()
for frame_id, frame in evaluation_frames_data.items():
    #print(frame['variants'])
    
    new_variants = []
    for var in frame['variants']:
        if len(var.split()) == 1:
            if RusLem.parse(var)[0].tag.POS in ['VERB', 'INFN']:
                new_variants.append(var)
            else:
                print(var, RusLem.parse(var)[0].tag.POS)
                
    #print(new_variants)
    if len(new_variants) > 0:
        new_frame = {
            'title': frame['title'],
            'variants': new_variants
        }
        word_set = word_set | set(new_variants)
    if len(new_variants) > 1:
        evaluation_frames_data_filter[frame_id] = new_frame
    all_word_cnt += len(frame['variants'])
    single_word_cnt += len(new_variants)
    
print(f'Всего оценочных фреймов c одним словом {len(evaluation_frames_data_filter)}')


print(f'Всего variants в оценочных фреймах {all_word_cnt}')
print(f'Всего variants из одного слова в оценочных фреймах {single_word_cnt}')

Всего оценочных фреймов: 311
надежда NOUN
расчет NOUN
ставка NOUN
перемирать NOUN
самоубийство NOUN
смерть NOUN
суицид NOUN
победа NOUN
успех NOUN
поддержка NOUN
подмога NOUN
покровительство NOUN
помощь NOUN
подспорье NOUN
шефство NOUN
осознание NOUN
понимание NOUN
разумение NOUN
уразумение NOUN
уяснение NOUN
признание NOUN
восторг NOUN
восхищение NOUN
ласка NOUN
любование NOUN
любовь NOUN
симпатия NOUN
оценивание NOUN
почитание NOUN
почтение NOUN
почтительность NOUN
уважение NOUN
вера NOUN
доверие NOUN
боязнь NOUN
испуг NOUN
мандраж NOUN
опасение NOUN
паника NOUN
перепуг NOUN
страх ADVB
трепет NOUN
несогласие NOUN
отказ NOUN
превышение NOUN
угроза NOUN
шантаж NOUN
наезд NOUN
подозрение NOUN
запрет NOUN
запрещение NOUN
воспрещение NOUN
табу NOUN
табуирование NOUN
закрытие NOUN
воспрещение NOUN
вето NOUN
вздорожание NOUN
дорожание NOUN
подорожание NOUN
удорожание NOUN
геноцид NOUN
избавление NOUN
извести NOUN
изничтожение NOUN
искоренение NOUN
истребление NOUN
наезд NOUN
обезглавливание

воздаяние NOUN
блокада NOUN
блокировка NOUN
окружение NOUN
осада NOUN
прорыв NOUN
запугивание NOUN
террор NOUN
терроризирование NOUN
устрашение NOUN
плач NOUN
рыдание NOUN
нюни NOUN
долготерпеливый ADJF
долготерпение NOUN
терпеливость NOUN
терпеливый ADJF
терпение NOUN
недоумение NOUN
подрыв NOUN
потрясение NOUN
сотрясение NOUN
встряска NOUN
потрясение NOUN
потрясенный PRTF
шок NOUN
вдохновение NOUN
вдохновленность NOUN
воодушевление NOUN
задор NOUN
окрыление NOUN
пыл NOUN
эйфория NOUN
энтузиазм NOUN
погребение NOUN
похороны NOUN
воспоминание NOUN
незабытый ADJF
памятуя GRND
память NOUN
дарение NOUN
одаривание NOUN
поздравление NOUN
захваливание NOUN
перехваливание NOUN
внушение NOUN
выволочка NOUN
выговаривание NOUN
головомойка NOUN
критика NOUN
нахлобучка NOUN
разгон NOUN
разнос NOUN
укор NOUN
упрек NOUN
облаивание NOUN
охаивание NOUN
очернение NOUN
хуление NOUN
ругань NOUN
сквернословие NOUN
доклад NOUN
рапорт NOUN
предостережение NOUN
сосуществование NOUN
перевоспитание NOUN
отважи

In [4]:
evaluation_frames_data_filter

{'0_0': {'title': ['надеяться', 'возлагать надежду'],
  'variants': ['ждать',
   'опереться',
   'опираться',
   'чаять',
   'уповать',
   'понадеяться',
   'надеяться',
   'рассчитывать']},
 '0_1': {'title': ['довериться', 'положиться'],
  'variants': ['ввериться',
   'вверяться',
   'довериться',
   'доверяться',
   'полагаться',
   'положиться']},
 '0_2': {'title': ['умереть', 'уйти из жизни'],
  'variants': ['вешаться',
   'вымереть',
   'вымирать',
   'задохнуться',
   'задыхаться',
   'застрелиться',
   'отравиться',
   'отравляться',
   'перемереть',
   'повеситься',
   'повымереть',
   'потравиться',
   'травиться',
   'убиться',
   'умереть',
   'умирать',
   'утопиться']},
 '0_3': {'title': ['дохнуть (умирать)'],
  'variants': ['помереть',
   'помирать',
   'загнуться',
   'окочуриться',
   'окочуриваться',
   'дохнуть',
   'околеть',
   'издыхать',
   'подыхать',
   'сдохнуть',
   'издохнуть',
   'околевать',
   'подохнуть']},
 '0_4': {'title': ['выиграть', 'получить приз'],

# Генерируем тестовый набор данных

In [ ]:
# with open("./evaluation_frame_verb_clusters.gold.txt", "w") as f:
    for frame_id, frame in evaluation_frames_data_filter.items():
        descr = " ".join(frame['title'])
        verbs = " ".join(frame['variants'])
        f.write(f"{frame_id}\t{descr}\t{verbs}\n")

In [6]:
len(evaluation_frames_data_filter)

286

In [11]:
word_set

{'пошутить',
 'любезничать',
 'восхищать',
 'воодушевление',
 'поладить',
 'укокошить',
 'сдобрить',
 'затосковать',
 'насаждение',
 'окрыление',
 'травма',
 'удовлетвориться',
 'восхваление',
 'заколебаться',
 'мандраж',
 'поломать',
 'соглашаться',
 'воссоздать',
 'засомневаться',
 'опорочить',
 'выдумывать',
 'спаять',
 'одергивать',
 'расправляться',
 'договариваться',
 'продвигать',
 'ябедничать',
 'саботаж',
 'преумножение',
 'осадить',
 'загрязнить',
 'основать',
 'зевнуть',
 'поаплодировать',
 'обтрепывать',
 'казнить',
 'глодать',
 'пререкаться',
 'злиться',
 'грабеж',
 'надувать',
 'выкрасть',
 'нытье',
 'соблазниться',
 'помощь',
 'уесть',
 'лупцовка',
 'разнести',
 'предотвращение',
 'подшучивать',
 'стучать',
 'околпачивать',
 'укоротиться',
 'загрязнять',
 'схватить',
 'неприязнь',
 'отнятие',
 'струхнуть',
 'утеряться',
 'хихиканье',
 'хватануть',
 'презентовать',
 'завладеть',
 'козырять',
 'дезорганизовывать',
 'выдворение',
 'досадовать',
 'посетовать',
 'зубоскальств

In [9]:
len(word_set)

3238

In [7]:
def filter_symbols(word):
    return re.sub(r"[^ЁёА-яA-Za-z]+", "", word)

def get_svo_context(sc_token):
    svo_triplets = []

    subject_list = []
    object_list = []
    for child in sc_token.children:
        if child.dep_ == 'nsubj':
            subject_list.append(child)
            
        if child.dep_ == 'obj':
            object_list.append(child)
            
        #print(subject_list, object_list)
    if len(subject_list) == 0 or len(object_list) == 0:
        return []
    
    for nsubj in subject_list:
        for obj in object_list:
            nsubj_str = filter_symbols(str(nsubj))
            sc_token_str = filter_symbols(str(sc_token))
            obj_str = filter_symbols(str(obj))
            if nsubj_str == '' or sc_token_str == '' or obj_str == '':
                continue
            svo_triplets.append(
                f"{nsubj_str} {sc_token_str} {obj_str}\t{nsubj.lemma_} {sc_token.lemma_} {obj.lemma_}"
            )
            
    return svo_triplets


def get_words_context_from_text(text, words_to_search):
    '''
    Возвращаем список предложений в которых встречаются слова из words_to_search
    и список троек SVO с этими словами
    '''
    found_words_contexts = []
    found_svo_triplets = []
    
    words_to_search = set(words_to_search)
    
    doc = nlp(text)
    for sentence in doc.sents:
        #print(sentence)
        #displacy.render(sentence, style='dep', options={'compact':True})
        for token in sentence:
            #print(token.lemma_)
            if token.lemma_ in words_to_search:
                found_words_contexts.append("\t".join([token.lemma_, str(sentence).replace('\n', ' ')]))
            
                if token.pos_ in ['AUX', 'VERB']:
                    svo_triplets = get_svo_context(token)
                    found_svo_triplets += svo_tripletsaraneum_none_fasttextcbow_300_5_2018.tgz
    return found_words_contexts, found_svo_triplets

In [9]:
def get_news_json_files(news_dir='./news2017_parsed/'):
    news_json_filenames = []
    month_dirs = os.listdir(news_dir)
    for month_dir in month_dirs:
        files = os.listdir(
            os.path.join(news_dir, month_dir)
        )
        news_json_filenames += [os.path.join(news_dir, month_dir, f) for f in files]
    return news_json_filenames

In [255]:
news_json_filenames = get_news_json_files()
print(len(news_json_filenames), news_json_filenames)

8979 ['./news2017_parsed/201707b/11072017-130941-R537842856_htm.json', './news2017_parsed/201707b/07072017-150356-R537517848_htm.json', './news2017_parsed/201707b/07072017-170132-R537527863_htm.json', './news2017_parsed/201707b/07072017-210417-R537543936_htm.json', './news2017_parsed/201707b/10072017-060117-R537718313_htm.json', './news2017_parsed/201707b/10072017-220420-R537787217_htm.json', './news2017_parsed/201707b/07072017-111222-R537498900_htm.json', './news2017_parsed/201707b/07072017-130238-R537506730_htm.json', './news2017_parsed/201707b/09072017-130923-R537666343_htm.json', './news2017_parsed/201707b/11072017-110600-R537830914_htm.json', './news2017_parsed/201707b/07072017-041043-R537472556_htm.json', './news2017_parsed/201707b/07072017-160203-R537521823_htm.json', './news2017_parsed/201707b/07072017-081352-R537485028_htm.json', './news2017_parsed/201707b/09072017-041000-R537639384_htm.json', './news2017_parsed/201707b/10072017-081035-R537724293_htm.json', './news2017_parsed/

In [233]:
news = list(json.load(open(news_json_filenames[0])).values())[1]

In [234]:
news

'Supreme2.Ru, 10.04.2017 15:15:52 - Как улучшить циркуляцию крови в руках и ногах и избежать нарушения кровообращения\nКак улучшить циркуляцию крови в руках и ногах и избежать нарушения кровообращения\nПлохая циркуляция крови в руках и ногах является нарушением, приводящим к ухудшению притока крови организме. Это происходит потому, что формирующиеся бляшки блокируют кровеносные сосуды, информирует supreme2.ru.\nЭто нарушение может произойти в результате различных причин, наиболее распространенные из которых: диабет, высокое кровяное давление, курение, повышенный уровень холестерина, преимущественно сидячий образ жизни, злоупотребление алкоголем, наркотиками и кофеином, неподходящие положения для рук или ног.\nКак самому себе помочь\nОдевайтесь тепло. Плотно закрывайте одеждой шею, запястья и лодыжки.\nНосите обувь на толстой подошве.\nНе употребляйте тяжелую жирную пищу. Лучше употреблять продукты с низким содержанием жира, поскольку именно из избыточного жира образуются бляшки, закупо

In [235]:
doc = nlp(news)

In [236]:
s = list(doc.sents)

In [243]:
s[6]


Не употребляйте тяжелую жирную пищу.

In [245]:
displacy.render(s[6], style='dep', options={'compact':True})

In [197]:
get_words_context_from_text(news, ['переключить', 'показать', 'центр'])

БезФормата.


безформата
.
Ru Астраханская область Астрахань, 26.04.2017 18:00:48 - Александр Жилкин и Иван Затевахин «поделились» красотами волжской дельты
Александр Жилкин и Иван Затевахин «поделились» красотами волжской дельты
Задать вопрос?


ru
астраханский
область
астрахань
,
26.04.2017
18:00:48
-
александр
жилкин
и
иван
затевахин
«
поделиться
"
красотами
волжской
дельта


александр
жилкин
и
иван
затевахин
«
поделиться
"
красотами
волжской
дельта


задать
вопрос
?
Александр Жилкин и Иван Затевахин «поделились» красотами волжской дельты Первое, что сделал губернатор Астраханской области Александр Жилкин , принимая сегодня в своем кабинете известного телеведущего Ивана Затевахина, приехавшего в Астрахань для участия в эколого-просветительском проекте « GREENкор », - подарил ему книгу фотографий о красотах Нижней Волги.


александр
жилкин
и
иван
затевахин
«
поделиться
"
красотами
волжской
дельта
один
,
что
сделать
губернатор
астраханский
область
александр
жилкин
,
принимать
сегодня
в
свой
кабинет
известный
телеведущего
иван
затевахина
,
приехать
в
астрахань
для
участие
в
эколого
-
просветительский
проект
«
greenкор
"
,
-
подарить
он
книга
фотография
о
красотах
нижний
волга
.
В ответ Иван Затевахин вручил губернатору диск со своим фильмом « Дельта, дарящая жизнь », снятым в Астраханском биосферном заповеднике.


в
ответ
иван
затевахин
вручить
губернатор
диск
со
свой
фильм
«
дельта
,
дарить
жизнь
"
,
снять
в
астраханский
биосферный
заповедник
.
Этот обмен сразу придал встрече неформальный характер.


...

пользоваться
присутствие
на
встреча
генеральный
директор
предприятие
«
газпром
добыча
астрахань
"
андрей
мельниченко
,
губернатор
поблагодарить
он
за
целенаправленный
поддержка
молодёжный
экологический
проект
,
в
тот
число
новый
,
предложить
каспийский
пресс
-
центр
,
идея
приобщить
подрастать
смена
журналист
к
большой
работа
по
защита
каспийский
осётр
.
Александр Жилкин рассказал Ивану Затевахину о своем собственном опыте борьбы с браконьерством, начиная с кризисных 90-х годов, о том, в частности, почему он десяток лет назад публично пообещал не есть черной икры до полной победы над браконьерством. «


александр
жилкин
рассказать
иван
затевахину
о
свой
собственный
опыт
борьба
с
браконьерство
,
начинать
с
кризисный
90-х
год
,
о
том
,
в
частность
,
почему
он
десяток
год
назад
публично
пообещать
не
есть
чёрный
икра
до
полный
победа
над
браконьерство
.
«
К сожалению, это обещание я держу до сих пор.


к
сожаление
,
этот
обещание
я
дерзить
до
сей
пора
.
К сожалению – потому что полностью справиться с браконьерством пока не удалось, хотя сегодня масштабы его сократились значительно», - сказал Александр Жилкин .


к
сожаление
–
потому
что
полностью
справиться
с
браконьерство
пока
не
удаться
,
хотя
сегодня
масштаб
его
сократиться
значительно
"
,
-
сказать
александр
жилкин
.




(['переключить\tПодчеркнув, что любит и часто смотрит телепрограмму « Диалоги о животных », Александр Жилкин сразу переключил внимание собеседника на свою родную природу, на экологические акции, которые он поддерживает как коренной астраханец и глава рыбного региона. «',
  'центр\tПользуясь присутствием на встрече генерального директора предприятия « Газпром добыча Астрахань » Андрея Мельниченко, губернатор поблагодарил его за целенаправленную поддержку молодежных экологических проектов, в том числе новой, предложенной Каспийским пресс-центром, идеи приобщить подрастающую смену журналистов к большой работе по защите каспийского осетра.'],
 ['Александр переключил внимание\tалександр переключить внимание'])

In [244]:
get_words_context_from_text('Не употребляйте тяжелую жирную пищу',
                            ['употреблять'])

(['употреблять\tНе употребляйте тяжелую жирную пищу'], [])

In [150]:
for sent in doc.sents:
    for token in sent:
        print (token.lemma_)

безформата
.
ru
год
.
москва
москва
,
26.04.2017
18:00:21
-
в
центр
на
флотский
28
апрель
состояться
поэтический
вечер
«
мы
–
легендарный
"


в
центр
на
флотский
28
апрель
состояться
поэтический
вечер
«
мы
–
легендарный
"


последний
новость
москва
по
тема
:
в
центр
на
флотский
28
апрель
состояться
поэтический
вечер
«
мы
–
легендарный
"
в
  
московский
многофункциональный
культурный
центр
(
ммкц
)
28
апрель
пройти
вечер
греческий
культура
.
17:07
26.04.2017
район
марфино
свао
москва
вечер
греческий
культура
состояться
на
  
территория
свао
28
апрель
.
мероприятие
пройти
в
  
московский
многофункциональный
культурный
центр
(
ммкц
)
.
17:04
26.04.2017
район
северный
медведково
свао
в
центр
на
флотский
28
апрель
состояться
поэтический
вечер
«
мы
–
легендарный
"
-
москва
в
  
культурный
центр
«
онежский
"
на
  
флотский
28
апрель
состояться
мероприятие
«
мы
  
—
легендарный
"
.
16:56
26.04.2017
головинский
район
сао
москва
в
  
четверг
,
27
апрель
,
в
  
китайский
культурный
центр
на
  
се

In [ ]:
news_dirname = '201704'
print(f"NEWS DIRNAME: {news_dirname}")
os.makedirs(os.path.join("./news2017_svo", news_dirname))
sent_filepath = os.path.join("./news2017_svo", news_dirname, "sampled_sentences.txt")
svo_filepath = os.path.join("./news2017_svo", news_dirname, "sampled_svo_triplets.txt")
news_json_filenames = [
    os.path.join("./news2017_parsed", news_dirname, f) 
    for f in os.listdir(os.path.join("./news2017_parsed", news_dirname))]
print(f"JSON файлов в директории: {len(news_json_filenames)}")

with open(sent_filepath, "w") as sentences_f:
    with open(svo_filepath, "w") as svo_f:
        found_sentences_cnt = 0
        fount_svo_triplets_cnt = 0
        for news_file_idx, news_json in enumerate(news_json_filenames[:]):
            print(f"START PREPARE FILE: {news_file_idx, news_json}")
            news_dump = json.load(open(news_json))
            for news_name, news_text in news_dump.items():
                found_words_contexts, found_svo_triplets = get_words_context_from_text(
                    news_text, word_set
                )
                found_sentences_cnt += len(found_words_contexts)
                fount_svo_triplets_cnt += len(found_svo_triplets)
                
                #print(found_words_con texts, found_svo_triplets)
                if len(found_words_contexts) > 0:
                    sentences_f.write("\n".join(found_words_contexts) + "\n")
                if len(found_svo_triplets) > 0:
                    svo_f.write("\n".join(found_svo_triplets) + "\n")
                
                sys.stderr.write(
                    f'\r{news_name}, {news_text[:10]}\t' + 
                    f'found svo triplets cnt: {fount_svo_triplets_cnt}, found context count: {found_sentences_cnt}'
                )

NEWS DIRNAME: 201704
JSON файлов в директории: 1478
START PREPARE FILE: (0, './news2017_parsed/201704/10042017-151354-R528251344_htm.json')


10042017-151452-R528252039.htm, Новая Газе	found svo triplets cnt: 1653, found context count: 10081

START PREPARE FILE: (1, './news2017_parsed/201704/03042017-151047-R527382315_htm.json')


03042017-151722-R527385230.htm, ГТРК Ставр	found svo triplets cnt: 2838, found context count: 17463

START PREPARE FILE: (2, './news2017_parsed/201704/09042017-060139-R528091530_htm.json')


...

12042017-170718-R528542023.htm, БезФормата	found svo triplets cnt: 1066312, found context count: 5744808

START PREPARE FILE: (616, './news2017_parsed/201704/20042017-140149-R529519063_htm.json')


20042017-140613-R529522824.htm, БезФормата	found svo triplets cnt: 1068229, found context count: 5754492

START PREPARE FILE: (617, './news2017_parsed/201704/01042017-140127-R527169074_htm.json')


01042017-140952-R527169388.htm, LipetskMed	found svo triplets cnt: 1068741, found context count: 5757340

In [262]:
with open("./news2017_sampled_context/sampled_sentences.txt", "w") as sentences_f:
    with open("./news2017_sampled_context/sampled_svo_triplets.txt", "w") as svo_f:
        found_sentences_cnt = 0
        fount_svo_triplets_cnt = 0
        for news_file_idx, news_json in enumerate(news_json_filenames[:]):
            print(f"START PREPARE FILE: {news_file_idx, news_json}")
            news_dump = json.load(open(news_json))
            for news_name, news_text in news_dump.items():
                found_words_contexts, found_svo_triplets = get_words_context_from_text(
                    news_text, word_set
                )
                found_sentences_cnt += len(found_words_contexts)
                fount_svo_triplets_cnt += len(found_svo_triplets)
                
                #print(found_words_contexts, found_svo_triplets)
                if len(found_words_contexts) > 0:
                    sentences_f.write("\n".join(found_words_contexts) + "\n")
                if len(found_svo_triplets) > 0:
                    svo_f.write("\n".join(found_svo_triplets) + "\n")
                
                sys.stderr.write(
                    f'\r{news_name}, {news_text[:10]}\t' + 
                    f'found svo triplets cnt: {fount_svo_triplets_cnt}, found context count: {found_sentences_cnt}')

START PREPARE FILE: (0, './news2017_parsed/201707b/11072017-130941-R537842856_htm.json')


11072017-131117-R537842343.htm, ФОКУС, 11.	found svo triplets cnt: 1095, found context count: 6804

KeyboardInterrupt: 

In [ ]:
with open("./global_text.txt") as f:
    with open("./global_text_contexts.txt", "w") as new_f:
        part_cnt = 0
        finded_context_cnt = 0
        while 1:
            if part_cnt > 100:
                break
            print(f'Corpus part count: {part_cnt}, finded context count: {finded_context_cnt}')
            line = f.read(1000)
            if len(line) == 0:
                break
            #print(len(contexts))
            
            contexts = search_words_context(line, word_set)
            finded_context_cnt += len(contexts)
            #print(contexts)"любовь"
            new_f.write("\n".join(contexts) + '\n')
            part_cnt += 1
            

In [101]:
contexts

['наступить\tгновение наступить новый год',
 'поздравить\tновый год давать поздравить друг друг',
 'пожелать\tпоздравить друг друг пожелать любовь и',
 'ворваться\tмера двое грабитель ворваться в банк',
 'помешать\tс деньги злоумышленник помешать подъехать патрульный',
 'подъехать\tденьги злоумышленник помешать подъехать патрульный машина',
 'увидеть\tпатрульный машина грабитель увидеть полицейский вернуться',
 'отпустить\tнесколько час бандит отпустить трое заложник',
 'пострадать\tосвобожденный физически не пострадать вместе с',
 'стоить\tна банковский парковка стоить 10 автомобилькомпания',
 'решить\tbiostar microtech очевидно решить быть в',
 'выпустить\tчисло первый производитель выпустить материнский плата']

In [97]:
doc = nlp(contexts[-1].split('\t')[1])
displacy.render(doc, style='dep', options={'compact':True})

In [125]:
doc = nlp("производитель выпустить материнский плата")
displacy.render(doc, style='dep', options={'compact':True})

In [128]:
displacy.render(doc, style='dep', options={'compact':True})

# Получаем набор уникальных семантических фреймов из новостей

In [8]:
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec


In [7]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [9]:
model = api.load("word2vec-google-news-300")
#model.most_similar("glass")

2020-09-08 20:30:49,989 : INFO : loading projection weights from /home/dana_zl/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2020-09-08 20:32:53,492 : INFO : loaded (3000000, 300) matrix from /home/dana_zl/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [11]:
model.wv.vocab

/home/dana_zl/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


{'</s>': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa6ff10>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa6fcd0>,
 'for': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa6fe10>,
 'that': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa6f890>,
 'is': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa6f150>,
 'on': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73090>,
 '##': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73110>,
 'The': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73190>,
 'with': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73210>,
 'said': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73250>,
 'was': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa732d0>,
 'the': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73350>,
 'at': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa733d0>,
 'not': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73450>,
 'as': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa734d0>,
 'it': <gensim.models.keyedvectors.Vocab at 0x7f2f4fa73550>,
 'be': <gen

In [23]:
svo_files_list = [
    "/home/dana_zl/WordNetPopulation/news2017_svo/201701/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201702/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201703/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201704/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201707a/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201707b/sampled_svo_triplets.txt",
    "/home/dana_zl/WordNetPopulation/news2017_svo/201707c/sampled_svo_triplets.txt"
]

uniq_svo = defaultdict(int)
for svo_file in svo_files_list:
    print(f"Start parse file {svo_file}")
    with open(svo_file) as f:
        for line_idx, line in enumerate(f):
            if line_idx % 1000 == 0:
                sys.stderr.write(f"\r Nubmer of uniq SVO: {len(uniq_svo)}")
            if len(line.split("\t")) == 1:
                print("bad row")
                continue
            svo = line.split("\t")[1]
            uniq_svo[svo] += 1

Start parse file /home/dana_zl/WordNetPopulation/news2017_svo/201701/sampled_svo_triplets.txt


 Nubmer of uniq SVO: 639969

Start parse file /home/dana_zl/WordNetPopulation/news2017_svo/201702/sampled_svo_triplets.txt


 Nubmer of uniq SVO: 1099009

Start parse file /home/dana_zl/WordNetPopulation/news2017_svo/201703/sampled_svo_triplets.txt


...

 Nubmer of uniq SVO: 2023883

Start parse file /home/dana_zl/WordNetPopulation/news2017_svo/201707b/sampled_svo_triplets.txt


 Nubmer of uniq SVO: 2099792

Start parse file /home/dana_zl/WordNetPopulation/news2017_svo/201707c/sampled_svo_triplets.txt


 Nubmer of uniq SVO: 2432267

In [30]:
with open("/home/dana_zl/WordNetPopulation/news2017_svo/news_svo.all.tsv", "w") as f:
    for svo, stat in uniq_svo.items():
        f.write(svo.replace(" ", '\t').strip() + f"\t{stat}\n")

In [29]:
len(uniq_svo)

2432580

In [26]:
sorted(uniq_svo.items(), key= operator.itemgetter(1), reverse=True)[:100]

[('вы найти ошибка\n', 16928),
 ('az представлять новость\n', 11615),
 ('вы видеть сообщение\n', 10840),
 ('вы забыть пароль\n', 10445),
 ('представитель принять участие\n', 8599),
 ('ru принять участие\n', 7504),
 ('президент подписать указ\n', 7373),
 ('президент подписать закон\n', 6088),
 ('заместитель принять участие\n', 5458),
 ('информация забыть пароль\n', 5351),
 ('суд принять решение\n', 5210),
 ('глава принять участие\n', 4815),
 ('человек принять участие\n', 4442),
 ('прокуратура провести проверка\n', 4441),
 ('глава подписать указ\n', 4369),
 ('вы найти сталь\n', 3769),
 ('губернатор провести которое\n', 3754),
 ('газета приветствовать републикации\n', 3441),
 ('председатель принять участие\n', 3418),
 ('руководитель принять участие\n', 3324),
 ('сотрудник задержать подозреваемый\n', 3321),
 ('сотрудник принять участие\n', 3263),
 ('полицейский задержать подозреваемый\n', 3066),
 ('трамп подписать указ\n', 2786),
 ('губернатор принять участие\n', 2749),
 ('сотрудник задерж

In [2]:
from gensim.models.fasttext import FastText

In [3]:
#from gensim.models import KeyedVectors
w2v = FastText.load("./triframes/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model", 
                        #binary=True,
                        #unicode_errors='ignore'
                       )

AttributeError: Can't get attribute 'FastTextKeyedVectors' on <module 'gensim.models.deprecated.keyedvectors' from '/home/dana_zl/anaconda3/lib/python3.7/site-packages/gensim/models/deprecated/keyedvectors.py'>

In [23]:
w2v = gensim.models.KeyedVectors.load(
    "./triframes/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model", 
                        #binary=True,
                        #unicode_errors='ignore'
                       )

In [20]:
w2v

NameError: name 'w2v' is not defined

In [4]:
import gensim

In [5]:
gensim.__version__

'3.8.3'

In [15]:
model_url = 'http://vectors.nlpl.eu/repository/11/181.zip'
m = wget.download(model_url)

KeyboardInterrupt: 

In [14]:
import wget